In [3]:
import os
from keys import OPENAI_API_KEY, SERPAPI_API_KEY, GOOGLE_API_KEY, GOOGLE_CSE_ID, WOLFRAM_ALPHA_APPID
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['GOOGLE_CSE_ID'] = GOOGLE_CSE_ID
os.environ['WOLFRAM_ALPHA_APPID'] = WOLFRAM_ALPHA_APPID

## LangChain Agents and Toolkits

In [8]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

/home/bharat/mambaforge/envs/llm-env/lib/python3.11/site-packages/langchain/llms/openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/bharat/mambaforge/envs/llm-env/lib/python3.11/site-packages/langchain/llms/openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### SERP API (Scraping Search Engines)

In [9]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(['serpapi', 'requests_all'], llm=llm, serpapi_api_key=SERPAPI_API_KEY)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [12]:
agent.run("tell me what is midjourney?")



> Entering new AgentExecutor chain...
I don't know what midjourney is, so I should search for it.
Action: Search
Action Input: "midjourney"
Observation: Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc. Midjourney generates images from natural language descriptions, called "prompts", similar to OpenAI's DALL-E and Stable Diffusion.
Thought:

OutputParserException: Could not parse LLM output: `Midjourney is a generative artificial intelligence program and service created by Midjourney, Inc. It generates images from natural language descriptions.`

### Google Search API

In [14]:
from langchain. utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Spain?", 3)

[{'title': 'Madrid | History, Population, Climate, & Facts | Britannica',
  'link': 'https://www.britannica.com/place/Madrid',
  'snippet': "Jul 24, 2023 ... Madrid, city, capital of Spain and of Madrid provincia (province). Spain's arts and financial centre, the city proper and province form a\xa0..."},
 {'title': 'Madrid - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Madrid',
  'snippet': 'Madrid is the capital and most populous city of Spain. The city has almost 3.4 million inhabitants and a metropolitan area population of approximately 6.7\xa0...'},
 {'title': 'Madrid - Eurocities',
  'link': 'https://eurocities.eu/cities/madrid/',
  'snippet': 'As the Spanish capital, Madrid is home to embassies and international organisations, major companies and financial institutions. It ranks first in Spain for the\xa0...'}]

In [13]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

In [7]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent("What is the national drink in Spain?") )



> Entering new AgentExecutor chain...
 I should research this online.
Action: google_search
Action Input: "national drink in Spain"
Observation: Sangría is Spain's national drink, so of course it belongs on this list! Spain is famous for it's wine, which is the base of this drink. Although there is nothing officially considered the national alcoholic drink of Spain, many people would say is the Spanish sherry wine (also known as vino de ... Apr 20, 2021 ... With a refreshing blend of fruits and spices, sangria is known to be the signature Spanish beverage. Ingredients can include ginger, cinnamon, ... "Very Old Rare Sherry", Pedro Ximenez by Garvey. Jerez de la Frontera (Andalusia, Spain), aged 30 years. Sherry is a national liquor of Spain. Nov 27, 2012 ... The fervor with which Spaniards drink gin & tonics is, to say the ... gin tonic is the national drink (and that ampersands are expendable). Even out of the country, people recognize the sherry wine as a very typical drink from Sp

### Requests / Web APIs

In [15]:
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

response = agent.run("Get the list of users at https://64df18b471c3335b25821167.mockapi.io/user and tell me the total number of users")



> Entering new AgentExecutor chain...
 I need to get the list of users from the website
Action: requests_get
Action Input: https://64df18b471c3335b25821167.mockapi.io/user
Observation: [{"name":"John Doe","email":"john.doe@example.com","id":"1"},{"name":"name 2","email":"email 2","id":"2"},{"name":"name 3","email":"email 3","id":"3"},{"name":"name 4","email":"email 4","id":"4"},{"name":"name 5","email":"email 5","id":"5"},{"name":"name 6","email":"email 6","id":"6"},{"name":"name 7","email":"email 7","id":"7"},{"name":"name 8","email":"email 8","id":"8"},{"name":"name 9","email":"email 9","id":"9"},{"name":"name 10","email":"email 10","id":"10"}]
Thought: I now know the total number of users
Final Answer: 10 users

> Finished chain.


### Python Shell / REPL Generation

In [16]:
tools = load_tools(["python_repl"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

print( agent.run("Create a list of random strings containing 4 letters, list should contain 30 examples, and sort the list alphabetically") )



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to generate a list of random strings, sort them, and then print the result
Action: Python_REPL
Action Input: 
import random

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: NameError("name 'string' is not defined")
Thought: I need to import the string module
Action: Python_REPL
Action Input: 
import random
import string

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: ['amwg', 'boxn', 'ciny', 'coso', 'dfui', 'guoa', 'gzul', 'htqg', 'isxr', 'iweo', 'lrzx', 'mclp', 'mpxn', 'nkuh', 'ohee', 'plpk', 'prmc', 'qvgt', 'raqp', 'shvw', 'skvr', 'twyv', 'ucpm', 'vcds', 'vozp', 'vtkg', 'xpyp', 'xyok', 'xzmw', 'xzvx']

Thought: I now know the final answer
Final Answer: ['amwg', 'boxn', 'ciny', 'coso', 'dfui', 'guoa', 'gzul', 'htqg', 'isxr', 'iweo', 'lrzx', 'mclp', 'mpxn', 'nkuh', 'ohee', 'pl

In [18]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

tools = load_tools(["wikipedia"])

print( agent.run("What is Nostradamus know for") )



> Entering new AgentExecutor chain...
 He is a famous French seer
Action: Python_REPL
Action Input: print("Nostradamus is known for his prophecies and predictions.")
Observation: Nostradamus is known for his prophecies and predictions.

Thought: I now know the final answer
Final Answer: Nostradamus is known for his prophecies and predictions.

> Finished chain.
Nostradamus is known for his prophecies and predictions.


### Wolfram Alpha

In [5]:
from langchain. utilities.wolfram_alpha import WolframAlphaAPIWrapper

wolfram = WolframAlphaAPIWrapper()
result = wolfram.run("What is 2x+5 = -3x + 7?")
print(result)

Assumption: 2 x + 5 = -3 x + 7 
Answer: x = 2/5


In [10]:
tools = load_tools(["wolfram-alpha"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent.run("How many days until the next Solar eclipse") )



> Entering new AgentExecutor chain...
I should use wolfram_alpha to find the answer to this question.
Action: wolfram_alpha
Action Input: "next solar eclipse"
Observation: Assumption: next | solar eclipse | after today 
Answer: Saturday, October 14, 2023 (2 months from now)
Thought:I now know the final answer
Final Answer: The next solar eclipse will occur on Saturday, October 14, 2023.

> Finished chain.
The next solar eclipse will occur on Saturday, October 14, 2023.


### Multi Tool (Collaboration) Example

In [17]:
tools = load_tools(["wolfram-alpha", "wikipedia"], llm=llm)

agent = initialize_agent(
		tools,
		llm,
		agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
		verbose=True
	)

agent.run("Who is Sachin Tendulkar's son? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
I need to find out who Sachin Tendulkar's son is and then calculate his current age raised to the 0.23 power.
Action: Wikipedia
Action Input: Sachin Tendulkar
Observation: Page: Sachin Tendulkar
Summary: Sachin Ramesh Tendulkar, ( (listen); pronounced [sətɕin teːɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest batsmen in the history of cricket. He is the all-time highest run-scorer in both ODI and Test cricket with more than 18,000 runs and 15,000 runs, respectively. He also holds the record for receiving the most man-of-the-match awards in international cricket. Sachin was a Member of Parliament, Rajya Sabha by nomination from 2012 to 2018.Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India i

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jmv25ZQj3Isy3eNyurbrenXP on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


KeyboardInterrupt: 